# 필수라이브러리 설치

In [1]:
import pandas as pd
import re
from docx import Document
import json
import os

from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import json
import os

from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from openai import OpenAI
import os

In [2]:
os.environ["OPENAI_API_KEY"] = "Key"


# 전처리 및 청킹

In [3]:
### --- 1. 조직도 전처리 --- ###

def parse_org_chart_multi_root(paragraphs):
    forest = []
    stack = []

    for line in paragraphs:
        indent_level = len(line) - len(line.lstrip('│ '))
        clean_line = line.lstrip('│ ').lstrip('├└─ ')
        
        match = re.match(r'(.+?) — (.+?) \((.+?)\)', clean_line)
        match_simple = re.match(r'(.+?) \((.+?)\)', clean_line)

        node = {}
        if match:
            title, name, position = match.groups()
            node = {'type': 'role', 'title': title.strip(), 'name': name.strip(), 'position': position.strip()}
        elif match_simple:
            name, position = match_simple.groups()
            node = {'type': 'person', 'name': name.strip(), 'position': position.strip()}
        else:
            node = {'type': 'unit', 'name': clean_line.strip()}

        node['children'] = []

        while stack and stack[-1]['indent'] >= indent_level:
            stack.pop()

        if stack:
            parent = stack[-1]['node']
            parent['children'].append(node)
        else:
            forest.append(node)

        stack.append({'indent': indent_level, 'node': node})

    return forest

def flatten_org_tree_to_chunks(tree, path=None):
    if path is None:
        path = []
    chunks = []

    for node in tree:
        current_text = ""
        metadata = {}

        if node["type"] == "unit":
            current_text = node["name"]
        elif node["type"] == "role":
            current_text = f"{node['title']} {node['name']} ({node['position']})"
            metadata = {"title": node["title"], "name": node["name"], "position": node["position"]}
        elif node["type"] == "person":
            current_text = f"{node['name']} ({node['position']})"
            metadata = {"name": node["name"], "position": node["position"]}

        full_path = path + [current_text]
        chunk_text = " > ".join(full_path)

        chunks.append({
            "text": chunk_text,
            "metadata": metadata
        })

        if node.get("children"):
            chunks.extend(flatten_org_tree_to_chunks(node["children"], path=full_path))

    return chunks

def process_org_chart():
    docx_path = "org_chart.docx"
    doc = Document(docx_path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    forest = parse_org_chart_multi_root(paragraphs)
    return flatten_org_tree_to_chunks(forest)

### --- 2. 인사정보 전처리 --- ###

def safe_str(val):
    if pd.isna(val):
        return ""
    return str(val).strip()

def process_hr_excel():
    xlsx_path = "HR information.xlsx"
    df = pd.read_excel(xlsx_path, sheet_name=0)
    person_chunks = []

    for _, row in df.iterrows():
        name = safe_str(row['성명'])
        dept = safe_str(row['부서'])
        pos = safe_str(row['직급'])
        join = safe_str(row['입사일'])       
        duty = safe_str(row['담당 업무'])
        eval_ = safe_str(row['최근 평가'])
        base = safe_str(row['기본급(₩)'])
        bonus = safe_str(row['성과급(₩)'])
        cert = safe_str(row['자격증·학위'])
        edu = safe_str(row['주요 교육·이수'])
        rr = safe_str(row['직무/책임 (R&R)'])

        text_parts = [
            f"{name}은(는) {dept} 부서의 {pos}입니다.",
            f"담당 업무는 {duty}이며, 최근 평가는 {eval_}입니다." if duty else "",
            f"기본급은 {base}원, 성과급은 {bonus}원입니다." if base or bonus else "",
            f"보유 자격증 및 학위: {cert}." if cert else "",
            f"이수 교육: {edu}." if edu else "",
            f"직무 책임(R&R): {rr}." if rr else ""
        ]
        full_text = " ".join([part for part in text_parts if part])
        metadata = {
            "이름": name,
            "부서": dept,
            "직급": pos,
            "입사일": join,
            "담당업무": duty,
            "평가": eval_,
            "기본급": base,
            "성과급": bonus,
            "자격증": cert,
            "교육": edu,
            "R&R": rr
        }

        person_chunks.append({"text": full_text, "metadata": metadata})
    return person_chunks

### --- 3. 실행 및 저장 --- ###

def run_all():
    org_chunks = process_org_chart()
    hr_chunks = process_hr_excel()
    combined = org_chunks + hr_chunks

    # ✅ 저장 경로 고정
    save_path = "output/combined_chunks.json"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(combined, f, ensure_ascii=False, indent=2)

    print(f"✅ 전처리 및 청크 완료: 총 {len(combined)}개 저장됨 → {save_path}")

### 실행
run_all()


✅ 전처리 및 청크 완료: 총 52개 저장됨 → output/combined_chunks.json


# 청크 확인

In [4]:
import json

with open("output/combined_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"총 청크 개수: {len(chunks)}개")




총 청크 개수: 52개


In [9]:
import random

for i, chunk in enumerate(random.sample(chunks, 20)):
    print(f"\n--- 샘플 {i+1} ---")
    print("📄 TEXT:\n", chunk.get("text", ""))
    print("📎 METADATA:\n", chunk.get("metadata", {}))



--- 샘플 1 ---
📄 TEXT:
 김재현은(는) 기술본부 부서의 CTO입니다. 담당 업무는 기술 로드맵·R&D이며, 최근 평가는 A0입니다. 기본급은 8200000원, 성과급은 2000000원입니다. 보유 자격증 및 학위: PhD CS. 이수 교육: AWS Architect. 직무 책임(R&R): 고객 피드백 대응, VOC 수집.
📎 METADATA:
 {'이름': '김재현', '부서': '기술본부', '직급': 'CTO', '입사일': '2019-06-03 00:00:00', '담당업무': '기술 로드맵·R&D', '평가': 'A0', '기본급': '8200000', '성과급': '2000000', '자격증': 'PhD CS', '교육': 'AWS Architect', 'R&R': '고객 피드백 대응, VOC 수집'}

--- 샘플 2 ---
📄 TEXT:
 서준호은(는) 선수관리팀 부서의 대리입니다. 담당 업무는 훈련 동행 및 장비 확인이며, 최근 평가는 B0입니다. 기본급은 3300000원, 성과급은 500000원입니다. 보유 자격증 및 학위: 없음. 이수 교육: 현장 오퍼레이션 트레이닝. 직무 책임(R&R): 장비 체크, 스케줄 현장 대응.
📎 METADATA:
 {'이름': '서준호', '부서': '선수관리팀', '직급': '대리', '입사일': '2024-01-05 00:00:00', '담당업무': '훈련 동행 및 장비 확인', '평가': 'B0', '기본급': '3300000', '성과급': '500000', '자격증': '없음', '교육': '현장 오퍼레이션 트레이닝', 'R&R': '장비 체크, 스케줄 현장 대응'}

--- 샘플 3 ---
📄 TEXT:
 CFO — 이재용 > Finance & Accounting 윤태오 (이사)
📎 METADATA:
 {'title': 'Finance & Accounting', 'name': '윤태오', 'position': '이사'}

--- 샘플 4 ---
📄 TEXT:
 정다온은(는) 전략기획팀 

In [5]:
lengths = [len(c["text"]) for c in chunks]
print(f"최소 길이: {min(lengths)}자 / 최대 길이: {max(lengths)}자 / 평균: {sum(lengths)//len(lengths)}자")


최소 길이: 5자 / 최대 길이: 185자 / 평균: 88자


In [11]:
from collections import Counter

def analyze_metadata(chunks):
    all_keys = []
    for chunk in chunks:
        keys = list(chunk.get("metadata", {}).keys())
        all_keys.extend(keys)
    return Counter(all_keys)

key_stats = analyze_metadata(chunks)
print("📊 메타데이터 키 사용 빈도:")
for k, v in key_stats.items():
    print(f"  - {k}: {v}개")


📊 메타데이터 키 사용 빈도:
  - name: 20개
  - position: 20개
  - title: 13개
  - 이름: 25개
  - 부서: 25개
  - 직급: 25개
  - 입사일: 25개
  - 담당업무: 25개
  - 평가: 25개
  - 기본급: 25개
  - 성과급: 25개
  - 자격증: 25개
  - 교육: 25개
  - R&R: 25개


In [6]:
from collections import Counter

text_counts = Counter(c["text"] for c in chunks)
duplicates = [text for text, count in text_counts.items() if count > 1]

print(f"⚠️ 중복된 청크 수: {len(duplicates)}개")
if duplicates:
    print("중복 예시:", duplicates[:3])


⚠️ 중복된 청크 수: 0개


In [7]:
keyword = "연구"  # 원하는 키워드로 변경 가능
filtered = [c for c in chunks if keyword in c["text"]]

print(f"🔎 '{keyword}' 포함된 청크 수: {len(filtered)}개")
for c in filtered[:3]:
    print("👉", c["text"])


🔎 '연구' 포함된 청크 수: 5개
👉 CTO — 김재현 > R&D 팀 > 김다인 (책임연구원)
👉 CTO — 김재현 > R&D 팀 > 정세윤 (주임연구원)
👉 김다인은(는) 연구개발 부서의 책임연구원입니다. 담당 업무는 모델 개발·파인튜닝이며, 최근 평가는 A0입니다. 기본급은 5000000원, 성과급은 1000000원입니다. 보유 자격증 및 학위: 정보처리기사. 이수 교육: NLP 심화. 직무 책임(R&R): 기술 로드맵 수립, R&D 관리, 모델 품질 보증.


# 임베딩 ( Vector DB 생성 )
1. Faiss DB를 이용
2. faiss_org / index.faiss

In [8]:
# ✅ 1. 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(
    model_name="models/KURE-V1",
    model_kwargs={"device": "cuda"},  # CPU 사용 시 "cpu"
    encode_kwargs={"normalize_embeddings": True}
)

# ✅ 2. combined_chunks.json 불러오기
with open("output/combined_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# ✅ 3. 문서 포맷으로 정리 (LangChain의 Document 타입 없이 dict로 충분)
texts = [c["text"] for c in chunks]
metadatas = [c.get("metadata", {}) for c in chunks]

# ✅ 4. FAISS 벡터 DB 생성
vectorstore = FAISS.from_texts(texts=texts, embedding=embedding_model, metadatas=metadatas)

# ✅ 5. 저장 경로 지정 및 저장
save_dir = "faiss_org_hr"
os.makedirs(save_dir, exist_ok=True)
vectorstore.save_local(folder_path=save_dir, index_name="index")

print(f"✅ FAISS 인덱스 저장 완료 → {save_dir}/index.faiss")


C:\Users\kdy\AppData\Local\Temp\ipykernel_25000\1509701770.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
c:\Users\kdy\anaconda3\envs\llm3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ FAISS 인덱스 저장 완료 → faiss_org_hr/index.faiss


# 평가지 생성
1. OpenAI API를 이용하여 각 항목에 대해1~2개 총 102개의 질문지 생성
2. gpt-4o 모델을 이용하여 정확한 평가지 작성성 

In [12]:

# 평가지 만들기 

import json
from openai import OpenAI
import os
import time

# ✅ OpenAI client (환경변수로 키 관리 중)
client = OpenAI()

# ✅ 1. 청크 불러오기
with open("output/combined_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# ✅ 2. 질문 생성 프롬프트 정의
def build_prompt(text):
    return f"""다음 텍스트에서 의미 기반으로 질의응답 쌍을 1~2개 만들어주세요.

텍스트:
{text}

형식: 
[
  {{"question": "질문 내용", "answer": "정답 내용"}}
]
"""

# ✅ 3. 질문 생성 함수 (GPT 호출)
def generate_qa_from_text(text):
    prompt = build_prompt(text)
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "당신은 평가용 질문지를 만드는 어시스턴트입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2
        )
        content = response.choices[0].message.content.strip()
        qa_pairs = json.loads(content)
        return qa_pairs
    except Exception as e:
        print("❌ 오류 발생:", e)
        return []

# ✅ 4. 전체 QA 생성 실행
qa_dataset = []

for i, chunk in enumerate(chunks):
    text = chunk["text"]
    qa_pairs = generate_qa_from_text(text)
    for qa in qa_pairs:
        if qa.get("question") and qa.get("answer"):
            qa_dataset.append(qa)
    time.sleep(0.5)  # 💡 rate limit 방지 (필요시 조정)

    if i % 10 == 0:
        print(f"🔄 {i}/{len(chunks)}개 처리 중...")

# ✅ 5. 저장
save_path = "output/eval_questions_gpt.jsonl"
with open(save_path, "w", encoding="utf-8") as f:
    for qa in qa_dataset:
        f.write(json.dumps(qa, ensure_ascii=False) + "\n")

print(f"✅ GPT 기반 질문지 {len(qa_dataset)}개 저장 완료 → {save_path}")


🔄 0/52개 처리 중...
🔄 10/52개 처리 중...
🔄 20/52개 처리 중...
🔄 30/52개 처리 중...
🔄 40/52개 처리 중...
🔄 50/52개 처리 중...
✅ GPT 기반 질문지 102개 저장 완료 → output/eval_questions_gpt.jsonl
